In [91]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [92]:
train=pd.read_csv('data/train.csv')
test=pd.read_csv('data/test.csv')
train

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60
5,5,1991-04-01 00:00:00,0,1,SB,58
6,6,1991-04-01 00:00:00,0,1,WB,26
7,7,1991-04-01 00:00:00,0,2,EB,31
8,8,1991-04-01 00:00:00,0,2,NB,49
9,9,1991-04-01 00:00:00,0,2,SB,46


In [93]:
train['year']=train['time'].map(lambda x: x[0:4]).astype(int)
train['month']=train['time'].map(lambda x: x[5:7]).astype(int)
train['date']=train['time'].map(lambda x: x[8:10]).astype(int)
train['hour']=train['time'].map(lambda x: x[11:13]).astype(int)
train['minute']=train['time'].map(lambda x: x[14:16]).astype(int)
#train[['year','month','date','hour','minute']]
train2=pd.get_dummies(train['direction'])
train=pd.concat([train,train2],axis=1)

In [94]:
test['year']=test['time'].map(lambda x: x[0:4]).astype(int)
test['month']=test['time'].map(lambda x: x[5:7]).astype(int)
test['date']=test['time'].map(lambda x: x[8:10]).astype(int)
test['hour']=test['time'].map(lambda x: x[11:13]).astype(int)
test['minute']=test['time'].map(lambda x: x[14:16]).astype(int)
#train[['year','month','date','hour','minute']]
test2=pd.get_dummies(test['direction'])
test=pd.concat([test,test2],axis=1)
test.describe()

,row_id,x,y,year,month,date,hour,minute,EB,NB,NE,NW,SB,SE,SW,WB
count,2340.000000,2340.000000,2340.000000,2340.0,2340.0,2340.0,2340.00000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000,2340.000000
mean,850004.500000,1.138462,1.630769,1991.0,9.0,30.0,17.50000,20.000000,0.184615,0.184615,0.107692,0.030769,0.184615,0.030769,0.107692,0.169231
std,675.644137,0.801649,1.089611,0.0,0.0,0.0,3.45279,16.333422,0.388068,0.388068,0.310058,0.172729,0.388068,0.172729,0.310058,0.375036
min,848835.000000,0.000000,0.000000,1991.0,9.0,30.0,12.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,849419.750000,0.000000,1.000000,1991.0,9.0,30.0,14.75000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,850004.500000,1.000000,2.000000,1991.0,9.0,30.0,17.50000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,850589.250000,2.000000,3.000000,1991.0,9.0,30.0,20.25000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,851174.000000,2.000000,3.000000,1991.0,9.0,30.0,23.00000,40.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [95]:
train_x=train.drop(['congestion','time','direction'],axis=1)
train_y=train['congestion']
test_x=test.drop(['time','direction'],axis=1)

In [96]:
kf=KFold(n_splits=10,shuffle=True,random_state=71)
tr_idx,va_idx=list(kf.split(train))[0]
tr_x,va_x=train_x.iloc[tr_idx],train_x.iloc[va_idx]
tr_y,va_y=train_y.iloc[tr_idx],train_y.iloc[va_idx]

In [97]:
import xgboost as xgb

In [98]:
dtrain=xgb.DMatrix(tr_x,label=tr_y)
dvalid=xgb.DMatrix(va_x,label=va_y)
dtest=xgb.DMatrix(test_x.values)
dtrain

In [99]:
#params={'objective':'binary:logistic','silent':1,'random_state':71}
params={'objective':'reg:squarederror','random_state':71}
num_round=500
watchlist=[(dtrain,'train'),(dvalid,'eval')]

In [100]:
model=xgb.train(params,dtrain,num_round,early_stopping_rounds=20,evals=watchlist,)

[0]	train-rmse:36.48986	eval-rmse:36.53669
[1]	train-rmse:27.24832	eval-rmse:27.28194
[2]	train-rmse:21.11216	eval-rmse:21.13906
[3]	train-rmse:17.31298	eval-rmse:17.33001
[4]	train-rmse:14.52789	eval-rmse:14.53793
[5]	train-rmse:13.03167	eval-rmse:13.03781
[6]	train-rmse:12.08313	eval-rmse:12.08252
[7]	train-rmse:11.57098	eval-rmse:11.56775
[8]	train-rmse:11.31689	eval-rmse:11.31389
[9]	train-rmse:10.87255	eval-rmse:10.87148
[10]	train-rmse:10.76752	eval-rmse:10.76679
[11]	train-rmse:10.60551	eval-rmse:10.60468
[12]	train-rmse:10.56804	eval-rmse:10.56823
[13]	train-rmse:10.52937	eval-rmse:10.53038
[14]	train-rmse:10.50549	eval-rmse:10.50671
[15]	train-rmse:10.47435	eval-rmse:10.47658
[16]	train-rmse:10.29754	eval-rmse:10.30301
[17]	train-rmse:10.26622	eval-rmse:10.27134
[18]	train-rmse:10.21821	eval-rmse:10.22537
[19]	train-rmse:10.17002	eval-rmse:10.17714
[20]	train-rmse:10.04811	eval-rmse:10.05740
[21]	train-rmse:10.00767	eval-rmse:10.01659
[22]	train-rmse:9.94330	eval-rmse:9.94979


In [101]:
from sklearn.metrics import mean_squared_error
va_pred=model.predict(dvalid)
score=np.sqrt(mean_squared_error(va_y,va_pred))
print(f'logloss:{score:4f}')

logloss:9.056824


In [102]:
pred=model.predict(xgb.DMatrix(test_x))

In [103]:
pred

array([34.28665  , 20.934126 , 32.353134 , ..., 56.729218 ,  3.7582304,
       27.58652  ], dtype=float32)

In [104]:
submit=pd.read_csv('data/sample_submission.csv')
submit['congestion']=pred

In [108]:
submit.to_csv('submission.csv',index=False)

In [106]:
pred.shape

(2340,)